In [10]:
import pandas as pd
import csv
from pyseed.seed_client import SeedClient
from datetime import datetime


# Load the CSV file.. Change the file path accordingly
file_path = '/Users/aatmannbc/Downloads/combined_meter_seed_export.csv'
data = pd.read_csv(file_path)



In [ ]:
# Connection parameters
connection_params = {
    "name": "chattanooga_solar_analysis",
    "base_url": "https://seedv2.lbl.gov", # Change if needed
    "username": "aatmann.bc@bison.howard.edu", # Change your user name
    "api_key": "",# Add your API Key
    "port": 443,
    "use_ssl": True
}
# Initialize the SEED client using the connection parameters
client = SeedClient(
    None,
    connection_params=connection_params
)
# Testing if client
print(client.get_org_by_name("City of Chattanooga",True))

In [ ]:
# list all properties
SeedClient.cycle_id = 574
properties = client.get_buildings()
# Get property_view_id for property
for property in properties:
    print(property["property_view_id"])


In [ ]:
# To create meter use this function. 
meter_response = client.get_or_create_meter(
            property_view_id=3326380, # Use property_view_id extracted above
            meter_type="Electric - Solar", # Has to be one of these models (https://github.com/SEED-platform/seed/blob/d75074fcd64736a7174d8ce39073761b7507e479/seed/models/meters.py#L12)
            source="Custom Entry",
            source_id="1"  # Convert source_id to string (Source Id can be custom string)
        )
print(f"Meter created for property ID {property_view_id}: {meter_response}")
# meter_response = client.get_or_create_meter(3420484,"Electric - Solar","Manual Entry","1")  # Convert source_id to string
print(f"Meter created for property ID {property_view_id}: {meter_response}")

In [ ]:
# Testing meter upload

payload = [{'start_time': '2023-01-01', 'end_time': '2023-01-28', 'reading': 286.4804296, 'source_unit': 'kBtu (Thousand BTU)', 'conversion_factor': 3.412}, {'start_time': '2023-02-01', 'end_time': '2023-02-28', 'reading': 351.8289158, 'source_unit': 'kBtu (Thousand BTU)', 'conversion_factor': 3.412}, {'start_time': '2023-03-01', 'end_time': '2023-03-28', 'reading': 325.1337338, 'source_unit': 'kBtu (Thousand BTU)', 'conversion_factor': 3.412}, {'start_time': '2023-04-01', 'end_time': '2023-04-28', 'reading': 263.2518614, 'source_unit': 'kBtu (Thousand BTU)', 'conversion_factor': 3.412}, {'start_time': '2023-05-01', 'end_time': '2023-05-28', 'reading': 289.5912453, 'source_unit': 'kBtu (Thousand BTU)', 'conversion_factor': 3.412}, {'start_time': '2023-06-01', 'end_time': '2023-06-28', 'reading': 355.4335411, 'source_unit': 'kBtu (Thousand BTU)', 'conversion_factor': 3.412}, {'start_time': '2023-07-01', 'end_time': '2023-07-28', 'reading': 391.9486149, 'source_unit': 'kBtu (Thousand BTU)', 'conversion_factor': 3.412}, {'start_time': '2023-08-01', 'end_time': '2023-08-28', 'reading': 405.7504439, 'source_unit': 'kBtu (Thousand BTU)', 'conversion_factor': 3.412}, {'start_time': '2023-09-01', 'end_time': '2023-09-28', 'reading': 355.272047, 'source_unit': 'kBtu (Thousand BTU)', 'conversion_factor': 3.412}, {'start_time': '2023-10-01', 'end_time': '2023-10-28', 'reading': 383.0824222, 'source_unit': 'kBtu (Thousand BTU)', 'conversion_factor': 3.412}, {'start_time': '2023-11-01', 'end_time': '2023-11-28', 'reading': 372.9683316, 'source_unit': 'kBtu (Thousand BTU)', 'conversion_factor': 3.412}, {'start_time': '2023-12-01', 'end_time': '2023-12-28', 'reading': 366.2829981, 'source_unit': 'kBtu (Thousand BTU)', 'conversion_factor': 3.412}]

client.upsert_meter_readings_bulk(3420482,17396,payload)

In [ ]:
# Use your own csv path
with open("/Users/aatmannbc/Downloads/combined_meter_seed_export.csv", mode='r') as file:
        csv_reader = csv.DictReader(file)
        
        # Iterate over each row in the CSV
        for row in csv_reader:
            print("-----------------------------------------------")
            property_view_id = row['property_unique_id']
            meter_id = row['meter_id']

            # Create the payload with meter readings for each month
            payload = []
            for i in range(1, 13):  # Loop through each month's column
                start_time = datetime(2023, i, 1).strftime("%Y-%m-%d")
                end_time = datetime(2023, i, 28).strftime("%Y-%m-%d")
                reading = row[f'ac_monthly{i}(kWh/Month)']

                # Build the dictionary for the current month's meter reading. 
                # The format for payload must be as below.
                meter_data = {
                    "start_time": start_time,
                    "end_time": end_time,
                    "reading": float(reading) if reading else 0.0,  # Convert reading to float
                    "source_unit": "kBtu (Thousand BTU)",
                    "conversion_factor": 3.412
                }
                payload.append(meter_data)
            
            add = client.upsert_meter_readings_bulk(property_view_id,meter_id,payload)
            print(add)




In [ ]:
# Iterate over each row in the DataFrame to create meters
# For easy access to ID
id_list = {} #{'Seed_id':'PM_id'}
for index, row in data.iterrows():
    property_view_id = row["property_view_id"]
    property_id=row["PM Property ID"]
    id_list[property_view_id]=str(property_id)
print(id_list)


In [ ]:
for key,value in id_list.items():
    meter_response = client.get_or_create_meter(
            property_view_id=key,
            meter_type="Electric - Solar",
            source="Manual Entry",
            source_id=value # Convert source_id to string
        )
    print(f"Meter created for property ID {property_view_id}: {meter_response}")

In [ ]:
# Finding the first Meter ID and looping it to the total number of buildings. 
# (Meter ID in SEED if only one meter is assigned is increamental)
for i in range(17396,17457):
    print(i)    # Add the printed value to corresponding property column.